This solution is adapted from: https://gis.stackexchange.com/questions/285499/how-to-split-multiband-image-into-image-tiles-using-rasterio/290059#290059

In [1]:
import os
from itertools import product
import rasterio as rio
from rasterio import windows
import numpy as np

In [2]:
def generate_tif_file_list(rootdir):
    # Description: Given root directory, returns list
    # of all .tif files
    # Adapted from https://stackoverflow.com/questions/31948528/python-loop-through-files-of-certain-extensions

    import os
    extensions = ('.tif')
    tif_file_list = []

    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            ext = os.path.splitext(file)[-1].lower()
            if ext in extensions:
                file_location = os.path.join(subdir, file)
                if '.tif' in file_location.lower(): # ensures ending is .tif
                    tif_file_list.append(file_location)

    return (tif_file_list)

In [3]:
rootdir = os.getcwd()

In [4]:
file_list = generate_tif_file_list(rootdir)
file_list

['C:\\Users\\bwbel\\Google Drive\\Dropbox\\Slum Project 2018\\digitalglobe_ftp\\JJC rasters rgb\\17SEP28060647-S3DS_R1C3-058472609020_01_P001_554_rgb.tif',
 'C:\\Users\\bwbel\\Google Drive\\Dropbox\\Slum Project 2018\\digitalglobe_ftp\\JJC rasters rgb\\17SEP28060647-S3DS_R2C2-058472609020_01_P001_246_rgb.tif',
 'C:\\Users\\bwbel\\Google Drive\\Dropbox\\Slum Project 2018\\digitalglobe_ftp\\JJC rasters rgb\\17SEP28060647-S3DS_R2C2-058472609020_01_P001_247_rgb.tif',
 'C:\\Users\\bwbel\\Google Drive\\Dropbox\\Slum Project 2018\\digitalglobe_ftp\\JJC rasters rgb\\17SEP28060647-S3DS_R2C2-058472609020_01_P001_248_rgb.tif',
 'C:\\Users\\bwbel\\Google Drive\\Dropbox\\Slum Project 2018\\digitalglobe_ftp\\JJC rasters rgb\\17SEP28060647-S3DS_R2C2-058472609020_01_P001_24_rgb.tif',
 'C:\\Users\\bwbel\\Google Drive\\Dropbox\\Slum Project 2018\\digitalglobe_ftp\\JJC rasters rgb\\17SEP28060647-S3DS_R2C2-058472609020_01_P001_250_rgb.tif',
 'C:\\Users\\bwbel\\Google Drive\\Dropbox\\Slum Project 2018\\dig

In [5]:
def get_tiles(ds, width, height):
    nols, nrows = ds.meta['width'], ds.meta['height']
    offsets = product(range(0, nols, width), range(0, nrows, height))
    big_window = windows.Window(col_off=0, row_off=0, width=nols, height=nrows)
    for col_off, row_off in  offsets:
        window =windows.Window(col_off=col_off, row_off=row_off, width=width, height=height).intersection(big_window)
        transform = windows.transform(window, ds.transform)
        yield window, transform

In [6]:
def create_image_tiles(input_filename, width, height):
    # Given input_filename for image and desired
    # height/width parameter, creates directory with
    # width x height image tiles
    
    # assume image is in current working directory
    in_path = os.getcwd()+"\\"
    
    # Create output_tiles subdirectory if it doesn't exist
    # and set out_path to this subdirectory
    if not os.path.isdir(in_path+"output_tiles"):
        os.mkdir(in_path+"output_tiles")
    out_path = in_path+"output_tiles"
    
    # define output filename uniquely by
    # image filename
    output_filename = input_filename+'_tile_{}-{}.tif'
    
    with rio.open(os.path.join(in_path, input_filename)) as inds:
        tile_width, tile_height = width, height
        
        meta = inds.meta.copy()
        
        for window, transform in get_tiles(inds, width, height):
            print(window)
            print(window.width)
            print(window.height)
            if (window.width==width) and (window.height==height):
                windowed_raster = inds.read(window=window)
                
                # Check that there is enough valid data
                if np.count_nonzero(windowed_raster) > 0.99*3*width*height:
                    meta['transform'] = transform
                    meta['width'], meta['height'] = window.width, window.height
                    outpath = os.path.join(out_path,output_filename.format(int(window.col_off), int(window.row_off)))
                    with rio.open(outpath, 'w', **meta) as outds:
                        outds.write(windowed_raster)

In [8]:
for single_tif_file in file_list:
    create_image_tiles(single_tif_file[len(rootdir)+1:], 80, 80)

Window(col_off=0, row_off=0, width=80, height=80)
80
80
Window(col_off=0, row_off=80, width=80, height=80)
80
80
Window(col_off=0, row_off=160, width=80, height=80)
80
80
Window(col_off=0, row_off=240, width=80, height=80)
80
80
Window(col_off=0, row_off=320, width=80, height=80)
80
80
Window(col_off=0, row_off=400, width=80, height=80)
80
80
Window(col_off=0, row_off=480, width=80, height=80)
80
80
Window(col_off=0, row_off=560, width=80, height=80)
80
80
Window(col_off=0, row_off=640, width=80, height=80)
80
80
Window(col_off=0, row_off=720, width=80, height=80)
80
80
Window(col_off=0, row_off=800, width=80, height=80)
80
80
Window(col_off=0, row_off=880, width=80, height=80)
80
80
Window(col_off=0, row_off=960, width=80, height=80)
80
80
Window(col_off=0, row_off=1040, width=80, height=80)
80
80
Window(col_off=0, row_off=1120, width=80, height=80)
80
80
Window(col_off=0, row_off=1200, width=80, height=80)
80
80
Window(col_off=0, row_off=1280, width=80, height=80)
80
80
Window(col_of

Window(col_off=320, row_off=2320, width=80, height=80)
80
80
Window(col_off=320, row_off=2400, width=80, height=80)
80
80
Window(col_off=320, row_off=2480, width=80, height=80)
80
80
Window(col_off=320, row_off=2560, width=80, height=22)
80
22
Window(col_off=400, row_off=0, width=80, height=80)
80
80
Window(col_off=400, row_off=80, width=80, height=80)
80
80
Window(col_off=400, row_off=160, width=80, height=80)
80
80
Window(col_off=400, row_off=240, width=80, height=80)
80
80
Window(col_off=400, row_off=320, width=80, height=80)
80
80
Window(col_off=400, row_off=400, width=80, height=80)
80
80
Window(col_off=400, row_off=480, width=80, height=80)
80
80
Window(col_off=400, row_off=560, width=80, height=80)
80
80
Window(col_off=400, row_off=640, width=80, height=80)
80
80
Window(col_off=400, row_off=720, width=80, height=80)
80
80
Window(col_off=400, row_off=800, width=80, height=80)
80
80
Window(col_off=400, row_off=880, width=80, height=80)
80
80
Window(col_off=400, row_off=960, width=

Window(col_off=720, row_off=1200, width=80, height=80)
80
80
Window(col_off=720, row_off=1280, width=80, height=80)
80
80
Window(col_off=720, row_off=1360, width=80, height=80)
80
80
Window(col_off=720, row_off=1440, width=80, height=80)
80
80
Window(col_off=720, row_off=1520, width=80, height=80)
80
80
Window(col_off=720, row_off=1600, width=80, height=80)
80
80
Window(col_off=720, row_off=1680, width=80, height=80)
80
80
Window(col_off=720, row_off=1760, width=80, height=80)
80
80
Window(col_off=720, row_off=1840, width=80, height=80)
80
80
Window(col_off=720, row_off=1920, width=80, height=80)
80
80
Window(col_off=720, row_off=2000, width=80, height=80)
80
80
Window(col_off=720, row_off=2080, width=80, height=80)
80
80
Window(col_off=720, row_off=2160, width=80, height=80)
80
80
Window(col_off=720, row_off=2240, width=80, height=80)
80
80
Window(col_off=720, row_off=2320, width=80, height=80)
80
80
Window(col_off=720, row_off=2400, width=80, height=80)
80
80
Window(col_off=720, row_

Window(col_off=1520, row_off=80, width=80, height=80)
80
80
Window(col_off=1520, row_off=160, width=80, height=80)
80
80
Window(col_off=1520, row_off=240, width=80, height=80)
80
80
Window(col_off=1520, row_off=320, width=80, height=80)
80
80
Window(col_off=1520, row_off=400, width=80, height=80)
80
80
Window(col_off=1520, row_off=480, width=80, height=80)
80
80
Window(col_off=1520, row_off=560, width=80, height=80)
80
80
Window(col_off=1520, row_off=640, width=80, height=80)
80
80
Window(col_off=1520, row_off=720, width=80, height=80)
80
80
Window(col_off=1520, row_off=800, width=80, height=80)
80
80
Window(col_off=1520, row_off=880, width=80, height=80)
80
80
Window(col_off=1520, row_off=960, width=80, height=80)
80
80
Window(col_off=1520, row_off=1040, width=80, height=80)
80
80
Window(col_off=1520, row_off=1120, width=80, height=80)
80
80
Window(col_off=1520, row_off=1200, width=80, height=80)
80
80
Window(col_off=1520, row_off=1280, width=80, height=80)
80
80
Window(col_off=1520, 

80
80
Window(col_off=240, row_off=400, width=80, height=80)
80
80
Window(col_off=240, row_off=480, width=80, height=80)
80
80
Window(col_off=240, row_off=560, width=80, height=29)
80
29
Window(col_off=320, row_off=0, width=9, height=80)
9
80
Window(col_off=320, row_off=80, width=9, height=80)
9
80
Window(col_off=320, row_off=160, width=9, height=80)
9
80
Window(col_off=320, row_off=240, width=9, height=80)
9
80
Window(col_off=320, row_off=320, width=9, height=80)
9
80
Window(col_off=320, row_off=400, width=9, height=80)
9
80
Window(col_off=320, row_off=480, width=9, height=80)
9
80
Window(col_off=320, row_off=560, width=9, height=29)
9
29
Window(col_off=0, row_off=0, width=80, height=80)
80
80
Window(col_off=0, row_off=80, width=80, height=80)
80
80
Window(col_off=0, row_off=160, width=80, height=80)
80
80
Window(col_off=0, row_off=240, width=80, height=24)
80
24
Window(col_off=80, row_off=0, width=80, height=80)
80
80
Window(col_off=80, row_off=80, width=80, height=80)
80
80
Window(co

...

Window(col_off=720, row_off=160, width=80, height=80)
80
80
Window(col_off=720, row_off=240, width=80, height=80)
80
80
Window(col_off=720, row_off=320, width=80, height=80)
80
80
Window(col_off=720, row_off=400, width=80, height=80)
80
80
Window(col_off=720, row_off=480, width=80, height=80)
80
80
Window(col_off=720, row_off=560, width=80, height=80)
80
80
Window(col_off=720, row_off=640, width=80, height=80)
80
80
Window(col_off=720, row_off=720, width=80, height=80)
80
80
Window(col_off=720, row_off=800, width=80, height=72)
80
72
Window(col_off=800, row_off=0, width=61, height=80)
61
80
Window(col_off=800, row_off=80, width=61, height=80)
61
80
Window(col_off=800, row_off=160, width=61, height=80)
61
80
Window(col_off=800, row_off=240, width=61, height=80)
61
80
Window(col_off=800, row_off=320, width=61, height=80)
61
80
Window(col_off=800, row_off=400, width=61, height=80)
61
80
Window(col_off=800, row_off=480, width=61, height=80)
61
80
Window(col_off=800, row_off=560, width=61, 

Window(col_off=0, row_off=720, width=80, height=80)
80
80
Window(col_off=0, row_off=800, width=80, height=80)
80
80
Window(col_off=0, row_off=880, width=80, height=31)
80
31
Window(col_off=80, row_off=0, width=80, height=80)
80
80
Window(col_off=80, row_off=80, width=80, height=80)
80
80
Window(col_off=80, row_off=160, width=80, height=80)
80
80
Window(col_off=80, row_off=240, width=80, height=80)
80
80
Window(col_off=80, row_off=320, width=80, height=80)
80
80
Window(col_off=80, row_off=400, width=80, height=80)
80
80
Window(col_off=80, row_off=480, width=80, height=80)
80
80
Window(col_off=80, row_off=560, width=80, height=80)
80
80
Window(col_off=80, row_off=640, width=80, height=80)
80
80
Window(col_off=80, row_off=720, width=80, height=80)
80
80
Window(col_off=80, row_off=800, width=80, height=80)
80
80
Window(col_off=80, row_off=880, width=80, height=31)
80
31
Window(col_off=160, row_off=0, width=80, height=80)
80
80
Window(col_off=160, row_off=80, width=80, height=80)
80
80
Wind

Window(col_off=0, row_off=240, width=80, height=80)
80
80
Window(col_off=0, row_off=320, width=80, height=80)
80
80
Window(col_off=0, row_off=400, width=80, height=80)
80
80
Window(col_off=0, row_off=480, width=80, height=80)
80
80
Window(col_off=0, row_off=560, width=80, height=76)
80
76
Window(col_off=80, row_off=0, width=80, height=80)
80
80
Window(col_off=80, row_off=80, width=80, height=80)
80
80
Window(col_off=80, row_off=160, width=80, height=80)
80
80
Window(col_off=80, row_off=240, width=80, height=80)
80
80
Window(col_off=80, row_off=320, width=80, height=80)
80
80
Window(col_off=80, row_off=400, width=80, height=80)
80
80
Window(col_off=80, row_off=480, width=80, height=80)
80
80
Window(col_off=80, row_off=560, width=80, height=76)
80
76
Window(col_off=160, row_off=0, width=80, height=80)
80
80
Window(col_off=160, row_off=80, width=80, height=80)
80
80
Window(col_off=160, row_off=160, width=80, height=80)
80
80
Window(col_off=160, row_off=240, width=80, height=80)
80
80
Wind

Window(col_off=0, row_off=640, width=80, height=80)
80
80
Window(col_off=0, row_off=720, width=80, height=80)
80
80
Window(col_off=0, row_off=800, width=80, height=6)
80
6
Window(col_off=80, row_off=0, width=80, height=80)
80
80
Window(col_off=80, row_off=80, width=80, height=80)
80
80
Window(col_off=80, row_off=160, width=80, height=80)
80
80
Window(col_off=80, row_off=240, width=80, height=80)
80
80
Window(col_off=80, row_off=320, width=80, height=80)
80
80
Window(col_off=80, row_off=400, width=80, height=80)
80
80
Window(col_off=80, row_off=480, width=80, height=80)
80
80
Window(col_off=80, row_off=560, width=80, height=80)
80
80
Window(col_off=80, row_off=640, width=80, height=80)
80
80
Window(col_off=80, row_off=720, width=80, height=80)
80
80
Window(col_off=80, row_off=800, width=80, height=6)
80
6
Window(col_off=160, row_off=0, width=80, height=80)
80
80
Window(col_off=160, row_off=80, width=80, height=80)
80
80
Window(col_off=160, row_off=160, width=80, height=80)
80
80
Window(

Window(col_off=560, row_off=720, width=80, height=80)
80
80
Window(col_off=560, row_off=800, width=80, height=80)
80
80
Window(col_off=560, row_off=880, width=80, height=80)
80
80
Window(col_off=560, row_off=960, width=80, height=80)
80
80
Window(col_off=560, row_off=1040, width=80, height=80)
80
80
Window(col_off=560, row_off=1120, width=80, height=22)
80
22
Window(col_off=640, row_off=0, width=80, height=80)
80
80
Window(col_off=640, row_off=80, width=80, height=80)
80
80
Window(col_off=640, row_off=160, width=80, height=80)
80
80
Window(col_off=640, row_off=240, width=80, height=80)
80
80
Window(col_off=640, row_off=320, width=80, height=80)
80
80
Window(col_off=640, row_off=400, width=80, height=80)
80
80
Window(col_off=640, row_off=480, width=80, height=80)
80
80
Window(col_off=640, row_off=560, width=80, height=80)
80
80
Window(col_off=640, row_off=640, width=80, height=80)
80
80
Window(col_off=640, row_off=720, width=80, height=80)
80
80
Window(col_off=640, row_off=800, width=80

## TODO: iterate through all .TIFF files in a directory, tiling them and adding to directory output_tiles